In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.wcs import WCS
import matplotlib.cm as cm
import re
from tqdm import tqdm


In [2]:
test_results = np.load("/home/acontreras/PRISM/resultados/delight_baseline_r/test_results.npz")
df_test = pd.read_csv("/home/acontreras/PRISM/data/SERSIC/df_test_delight.csv")

In [32]:
def obtain_predicted_ra_dec(df_test, resultado):
    """
    Convierte predicciones de píxeles a coordenadas astronómicas (RA, Dec).

    Para cada ejemplo en el DataFrame de test, convierte las predicciones de
    posición (en píxeles) a coordenadas de mundo (RA, Dec) usando WCS.

    Parameters
    ----------
    df_test : pandas.DataFrame
        DataFrame con la información de test. Debe contener una columna "sn_coords" 
        con las coordenadas de la supernova en formato "(ra, dec)".
    resultados : dict
        Diccionario de resultados donde cada clave es el nombre de un experimento y 
        cada valor es una tupla (preds, targets, mean_preds, original_target).
    experiment_name : str
        Nombre del experimento cuyas predicciones se desean convertir.

    Returns
    -------
    np.ndarray
        Array de forma (N, 2) con las coordenadas (RA, Dec) predichas para cada ejemplo.
    """
    def get_wcs(ra, dec):
        wcs = WCS(naxis=2)
        wcs.wcs.crpix = [15.0, 15.0]
        wcs.wcs.cdelt = [-6.944444445183e-5, 6.944444445183e-5]  # grados/píxel
        wcs.wcs.crval = [ra, dec]
        wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]
        wcs.wcs.cunit = ["deg", "deg"]
        return wcs

    mean_preds = resultado['mean_preds']

    coordinates = []
    for idx in tqdm(range(len(df_test))):
    
        match = re.search(r'\(([-+]?(?:\d*\.\d+|\d+\.?)),\s*([-+]?(?:\d*\.\d+|\d+\.?))\)', df_test.iloc[idx]["sn_coords"])
        ra = np.float64(match.group(1))
        dec = np.float64(match.group(2))

        wcs = get_wcs(ra, dec)

        ra_host, dec_host = wcs.pixel_to_world_values([mean_preds[idx] +14])[0] # pred = (x,y) = (dec, ra)

        coordinates.append([ra_host, dec_host])
    
    return np.array(coordinates)

In [34]:
coords_preds = obtain_predicted_ra_dec(df_test, test_results)

100%|██████████| 4788/4788 [00:01<00:00, 3807.52it/s]


In [35]:
coords_preds

array([[120.51646582,  19.32178148],
       [155.64750255,  43.84704204],
       [240.98575232,  65.6715888 ],
       ...,
       [111.80696467,  85.75466934],
       [293.79283703,  87.27501888],
       [175.16727116,  89.08495575]])

In [36]:
df = pd.DataFrame(coords_preds, columns=['ra_pred','dec_pred'])

In [37]:
df

,ra_pred,dec_pred
0,120.516466,19.321781
1,155.647503,43.847042
2,240.985752,65.671589
3,154.163609,34.547588
4,243.214079,39.051648
...,...,...
4783,108.579117,84.382291
4784,121.105888,84.641370
4785,111.806965,85.754669
4786,293.792837,87.275019
